In [ ]:
from google.colab import drive
drive.mount('./gdrive')

In [ ]:
!ls

In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torchvision import datasets, transforms


In [ ]:
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.fc1 = nn.Linear(784,512)
        self.fc2 = nn.Linear(512,256)
        self.fc3 = nn.Linear(256,128)
        self.fc4 = nn.Linear(128,64)
        self.fc5 = nn.Linear(64,32)
        self.fc6 = nn.Linear(32,10)

    def forward(self, x):
        x = x.float()
        h1 = F.relu(self.fc1(x.view(-1, 784)))
        h2 = F.relu(self.fc2(h1))
        h3 = F.relu(self.fc3(h2))
        h4 = F.relu(self.fc4(h3))
        h5 = F.relu(self.fc5(h4))
        h6 = self.fc6(h5)
        return F.log_softmax(h6, dim=1)
    


In [ ]:
batch_size = 64
test_batch_size = 1000
epochs = 10
lr = .01
momentum = .5
seed = 1
log_interval = 200

torch.manual_seed(seed)
device = torch.device("cuda")
kwargs = {'num_workers': 1, 'pin_memory': True}

In [ ]:
transform = transforms.Compose([
                                transforms.ToTensor(),
                                transforms.Normalize((0.1307,), (0.3081,))])
train_loader = torch.utils.data.DataLoader(
    datasets.MNIST('../data', train=True, download=True, transform=transform),
    batch_size = batch_size, shuffle=True, **kwargs)



test_loader = torch.utils.data.DataLoader(
    datasets.MNIST('../data', train=False,download=True,transform=transform),
    batch_size=test_batch_size, shuffle=True, **kwargs
)

In [ ]:
model = Net()
optimizer = optim.SGD(model.parameters(), lr = lr, momentum=momentum)

In [ ]:
def train(log_interval, model, device, train_loader, optimizer, epoch):
    model.train()
    for batch_idx, (data, target) in enumerate(train_loader):
        data, target = data, target
        optimizer.zero_grad()
        output = model(data)
        loss = F.nll_loss(output, target)
        loss.backward()
        optimizer.step()
        if batch_idx%log_interval ==0:
            print('train epoch: {} [{}/{} ({:.0f}%)]\tLoss: {:.6f}'.format(
                epoch, batch_idx*len(data), len(train_loader.dataset),
                100.*batch_idx / len(train_loader), loss.item()))
def test(log_interval, model, device, test_loader):
    model.eval()
    test_loss =0
    correct =0
    with torch.no_grad():
        for data, target in test_loader:
            data, target = data, target
            ouput = model(data)
            test_loss += F.nll_loss(output, target, reduction='sum').item()
            pred = output.argmax(dim=1, keepdim=True)
            correct += pred.eq(target.view_as(pred)).sum().item()

    test_loss /= len(test_loader.dataset)

    print('\nTest set : average loss: {:.4f}, accuract: {}/{} ({:.0f}%)\n'.format(test_loss, correct, len(test_loader.dataset),
                                                                                  100.* correct / len(test_loader.dataset)))

In [ ]:
for epoch in range(1, 11):
    train(log_interval, model, device, train_loader, optimizer, epoch)
    test(log_interval, model, device, test_loader)

torch.save(model,'./model_mnist.pt')

Model Load

In [ ]:
path = "./model_mnist.pt"
model = torch.load(path)
model.eval()
model()

In [ ]:
    test_loss =0
    correct =0
    
    for data, target in test_loader:
        data, target = data, target
        output = model(data)
        test_loss += F.nll_loss(output, target, reduction='sum').item()
        pred = output.argmax(dim=1, keepdim=True)
        correct += pred.eq(target.view_as(pred)).sum().item()
        break

In [ ]:
correct = 0
total = 0
model.eval()
with torch.no_grad():
  for image,label in test_loader:
      x = image#.to(device)
      y_= label#.to(device)
      output = model.forward(x)
      _,output_index = torch.max(output, 1)
      total += label.size(0)
      correct += (output_index == y_).sum().float()
  print("Accuracy of Test Data: {}%".format(100.0*correct/total))